# Code for the Telegram Bot

In [ ]:
import asyncio
import nest_asyncio
import torch
from transformers import AutoModelForCausalLM, AutoTokenizer
from telegram import Update
from telegram.ext import ApplicationBuilder, CommandHandler, MessageHandler, ContextTypes, filters

# Apply the patch for running in environments with existing event loops
nest_asyncio.apply()


MODEL_PATH = r"U:\E.Mobility\WS 2024-25\Data Science Survival Skills\Homeworks\H9\TinyLlama-1.1B-Chat-v1.0"

# Load the LLM model and tokenizer
print("Loading model...")
try:
    tokenizer = AutoTokenizer.from_pretrained(MODEL_PATH)
    model = AutoModelForCausalLM.from_pretrained(MODEL_PATH)
    device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
    model.to(device)
    print("Model loaded successfully.")
except Exception as e:
    print(f"Error loading model: {e}")
    exit()

TOKEN = "7667178361:AAFRZFZqXl6kAYa5HXSSK_HcJixusnYiXbw"

# Function to handle incoming messages
async def handle_message(update: Update, context: ContextTypes.DEFAULT_TYPE):
    user_message = update.message.text.strip()
    user_id = update.effective_user.id

    print(f"Message from {user_id}: {user_message}")

    # Add system instruction to guide the LLM
    prompt = (
        "You are a knowledgeable assistant. Please provide detailed and accurate "
        "information about the topic asked.\n\nUser: " + user_message + "\nAssistant:"
    )
    inputs = tokenizer.encode(prompt, return_tensors="pt").to(device)

    try:
        # Generate response with controlled decoding
        outputs = model.generate(
            inputs,
            max_length=200,
            num_return_sequences=1,
            do_sample=True,
            temperature=0.7,
            top_k=50,
            top_p=0.9,
        )
        full_response = tokenizer.decode(outputs[0], skip_special_tokens=True)

        # Remove the prompt and ensure only the assistant's direct response is sent
        if "Assistant:" in full_response:
            bot_response = full_response.split("Assistant:")[-1].strip()
        else:
            bot_response = full_response.strip()

        # Stop at the first instance of "User:" in the response (if hallucinated)
        if "User:" in bot_response:
            bot_response = bot_response.split("User:")[0].strip()

    except Exception as e:
        print(f"Error during LLM inference: {e}")
        bot_response = "Sorry, I encountered an error while processing your request."

    # Send the response back to the user
    await update.message.reply_text(bot_response)

# Function to handle the /start command
async def start_command(update: Update, context: ContextTypes.DEFAULT_TYPE):
    await update.message.reply_text(
        "Welcome to Noob1.0! You can send me a message, and I'll try to respond."
    )

# Main function to start the bot
async def main():
    # Initialize the bot application
    app = ApplicationBuilder().token(TOKEN).build()

    # Add command and message handlers
    app.add_handler(CommandHandler("start", start_command))
    app.add_handler(MessageHandler(filters.TEXT & ~filters.COMMAND, handle_message))

    # Start the bot
    print("Bot is running...")
    await app.run_polling()

# Entry point of the script
if __name__ == "__main__":
    asyncio.run(main())


Loading model...
Model loaded successfully.
Bot is running...
Message from 7210772876: Tell me something about Lions
Message from 7210772876: Thanks


### For Screenshot Purpose
To add screenshots to homework file

In [ ]:
# import asyncio
# import nest_asyncio
# from telegram import Update
# from telegram.ext import ApplicationBuilder, CommandHandler, MessageHandler, ContextTypes, filters

# # Apply the patch for running in environments with existing event loops
# nest_asyncio.apply()

# TOKEN = "7667178361:AAFRZFZqXl6kAYa5HXSSK_HcJixusnYiXbw"

# async def handle_message(update: Update, 
#                          context: ContextTypes.DEFAULT_TYPE):
#     user_message = update.message.text.strip()
#     user_id = update.effective_user.id

#     print(f"Message from {user_id}: {user_message}")

# async def start_command(update: Update, 
#                         context: ContextTypes.DEFAULT_TYPE):
#     await update.message.reply_text(
#         "Welcome to Noob1.0! You can send me a message,\
#             and I'll try to respond."
#     )
# async def main():
#     app = ApplicationBuilder().token(TOKEN).build()

#     app.add_handler(CommandHandler("start", start_command))
#     app.add_handler(MessageHandler(
#         filters.TEXT & ~filters.COMMAND, handle_message))
#     print("Bot is running...")
#     await app.run_polling()

# if __name__ == "__main__":
#     asyncio.run(main())


In [ ]:
# import torch
# from transformers import AutoModelForCausalLM, AutoTokenizer

# MODEL_PATH = r"U:\E.Mobility\WS 2024-25\Data Science Survival \
#                 Skills\Homeworks\H9\TinyLlama-1.1B-Chat-v1.0"

# # Load the LLM model and tokenizer
# print("Loading model...")
# try:
#     tokenizer = AutoTokenizer.from_pretrained(MODEL_PATH)
#     model = AutoModelForCausalLM.from_pretrained(MODEL_PATH)
#     device = torch.device(
#         "cuda" if torch.cuda.is_available() else "cpu"
#     )
#     model.to(device)
#     print("Model loaded successfully.")
# except Exception as e:
#     print(f"Error loading model: {e}")
#     exit()

# def generate_response(user_message: str):
#     prompt = (
#         "You are a knowledgeable assistant. Please "
#         "provide detailed and accurate information about "
#         "the topic asked.\n\nUser: " + user_message + "\nAssistant:"
#     )
#     inputs = tokenizer.encode(prompt, return_tensors="pt").to(device)

#     try:
#         outputs = model.generate(inputs, max_length=200, 
#                                  num_return_sequences=1,do_sample=True, 
#                                  temperature=0.7, top_k=50, top_p=0.9)
#         full_response = tokenizer.decode(
#             outputs[0], skip_special_tokens=True)
#         bot_response = full_response.split("Assistant:")[-1].strip()
#         if "User:" in bot_response:
#             bot_response = bot_response.split("User:")[0].strip()

#     except Exception as e:
#         print(f"Error during LLM inference: {e}")
#         bot_response = (
#             "Sorry, I encountered an error while "
#             "processing your request."
#         )
#     return bot_response
